In [ ]:
from yugiquery import *

init_notebook_mode(all_interactive=True)

header("Bandai")

---

Table of Contents <a class="jp-toc-ignore"></a>
=================

*   [1  Data aquisition](#Data-aquisition)
    *   [1.1  Fetch online data](#Fetch-online-data)
*   [2  Check changes](#Check-changes)
    *   [2.1  Load previous data](#Load-previous-data)
    *   [2.2  Generate changelog](#Generate-changelog)
    *   [2.3  Save data](#Save-data)
*   [3  Data visualization](#Data-visualization)
    *   [3.1  Card type](#Card-type)
    *   [3.2  ATK](#ATK)
    *   [3.3  DEF](#DEF)
    *   [3.4  Level](#Level)
        *   [3.4.1  ATK statistics](#ATK-statistics)
        *   [3.4.2  DEF statistics](#DEF-statistics)
        *   [3.4.3  Level](#Level)
    *   [3.5  Monster type](#Monster-type)
    *   [3.6  Set](#Set)
*   [4  Epilogue](#Epilogue)
    *   [4.1  HTML export](#HTML-export)
<!--     *   [4.2  Git](#Git) -->

# Data aquisition

## Fetch online data

In [ ]:
timestamp = arrow.utcnow()

In [ ]:
bandai_df = fetch_bandai()

# Check changes

## Load previous data

In [ ]:
# Get latest file if exist
previous_df, previous_ts = load_corrected_latest("bandai")

if previous_df is not None:
    previous_df = previous_df.astype(
        bandai_df[previous_df.columns.intersection(bandai_df.columns)].dtypes.to_dict()
    )

## Generate changelog

In [ ]:
if previous_df is None:
    changelog = None
    print("Skipped")
else:
    changelog = generate_changelog(previous_df, bandai_df, col="Name")
    if not changelog.empty:
        display(changelog)
        changelog.to_csv(
            f'../data/{make_filename(report = "bandai", timestamp = timestamp, previous_timestamp = previous_ts)}',
            index=True,
        )
        print("Changelog saved")

## Save data

In [ ]:
if changelog is not None and changelog.empty:
    print("No changes. New data not saved")
else:
    bandai_df.to_csv(
        f'../data/{make_filename(report = "bandai", timestamp=timestamp)}', index=False
    )
    print("Data saved")

# Data visualization

In [ ]:
bandai_df

## Card type

In [ ]:
print("Total number of card types:", bandai_df["Card type"].nunique())

In [ ]:
card_type_colors = [colors_dict[i] for i in bandai_df["Card type"].value_counts().index]
bandai_df["Card type"].value_counts().plot.bar(
    figsize=(18, 6), grid=True, rot=0, color=card_type_colors
)
plt.show()

## ATK

In [ ]:
print("Total number of ATK values:", bandai_df["ATK"].nunique())

In [ ]:
bandai_df.drop(columns=["Card type", "Page name", "Page URL"]).groupby(
    "ATK"
).nunique().sort_index(key=lambda x: pd.to_numeric(x, errors="coerce"))

In [ ]:
atk_colors = colors_dict["Monster Card"]
bandai_df["ATK"].value_counts().sort_index(
    key=lambda x: pd.to_numeric(x, errors="coerce")
).plot.bar(figsize=(18, 6), grid=True, color=atk_colors)
plt.show()

## DEF

In [ ]:
print("Total number of DEF values:", bandai_df["DEF"].nunique())

In [ ]:
bandai_df.drop(columns=["Card type", "Page name", "Page URL"]).groupby(
    "DEF"
).nunique().sort_index(key=lambda x: pd.to_numeric(x, errors="coerce"))

In [ ]:
def_colors = colors_dict["Monster Card"]
bandai_df["DEF"].value_counts().sort_index(
    key=lambda x: pd.to_numeric(x, errors="coerce")
).plot.bar(figsize=(18, 6), grid=True, color=def_colors)
plt.show()

## Level

In [ ]:
bandai_df.drop(columns=["Card type", "Page name", "Page URL"]).groupby(
    "Level"
).nunique().sort_index(key=lambda x: pd.to_numeric(x, errors="coerce"))

In [ ]:
level_colors = colors_dict["Level"]
bandai_df["Level"].value_counts().sort_index(
    key=lambda x: pd.to_numeric(x, errors="coerce")
).plot.bar(figsize=(18, 6), grid=True, rot=0, color=level_colors)
plt.show()

### ATK statistics

In [ ]:
bandai_df[["Level", "ATK"]].apply(pd.to_numeric, errors="coerce").dropna().astype(
    int
).groupby("Level").describe().round(1)

### DEF statistics

In [ ]:
bandai_df[["Level", "DEF"]].apply(pd.to_numeric, errors="coerce").dropna().astype(
    int
).groupby("Level").describe().round(1)

## Monster type

In [ ]:
print("Total number of monster types:", bandai_df["Monster type"].nunique())

In [ ]:
bandai_df.drop(columns=["Card type", "Page name", "Page URL"]).groupby(
    "Monster type"
).nunique()

In [ ]:
monster_type_colors = colors_dict["Monster Card"]
bandai_df["Monster type"].value_counts().plot.bar(
    figsize=(18, 6), grid=True, rot=45, color=monster_type_colors
)
plt.show()

## Set

In [ ]:
print("Total number of sets:", bandai_df["Set"].nunique())

In [ ]:
bandai_df.drop(columns=["Page name", "Page URL"]).groupby("Set").nunique()

# Epilogue

In [ ]:
benchmark(report="bandai", timestamp=timestamp)

In [ ]:
footer()

## HTML export

In [ ]:
# Save notebook on disck before generating HTML report
save_notebook()

In [ ]:
! jupyter nbconvert Bandai.ipynb --output-dir='../' --to=HTML --template=labdynamic --no-input

## Git

In [ ]:
commit("*[Bb]andai*", f"Bandai update - {timestamp.isoformat()}")